In [8]:
#List of cities for analysis
#Working

import pandas as pd
import numpy as np
import netCDF4 as nc
from datetime import datetime, timedelta

#Read city location data
city_location = pd.read_csv("worldcities_locations.csv")
city_location.head()

#Total unique cities (to be sure)
unique_city = np.unique(city_location.city)

#Sample 100 cities from total cities
city_100 = np.random.choice(unique_city, 100, replace = False)
#print(city_100)

#Final list of cities which will be used for analysis
final_cities = city_location[city_location['city'].isin(city_100)]
final_cities.head()
#print(final_cities.city.unique())
#print(len(final_cities.city.unique()))

,city,lat,lng,country,population
45,Belo Horizonte,-19.9150,-43.9150,Brazil,5575000
65,Dongguan,23.0489,113.7447,China,4528000
134,Campinas,-22.9000,-47.1000,Brazil,2791000
142,Lucknow,26.8550,80.9150,India,2695000
212,Indore,22.7151,75.8650,India,2026000


In [14]:
#Extract data for all cities from netcdf file in a dataframe
#working

#Set file name
file_path = 'gistemp1200_GHCNv4_ERSSTv5.nc' 

#Open netCDF file
dataset = nc.Dataset(file_path)

#Read the latitude, longitude, time and surface temperature data
lat = dataset.variables['lat'][:]
lon = dataset.variables['lon'][:]
temp = dataset.variables['tempanomaly'][:]
#Convert blank values to zero
temp[np.ma.getmaskarray(temp)] = 0
time = dataset.variables['time'][:]
units = dataset.variables['time'].units

#Set start and end date for which the data is present in netcdf file
starting_date = dataset.variables['time'].units[11:21]
ending_date = '2023-05-15'

#Create range of date as per values present in netcdf file (montly data for each year and locations)
date_range = pd.date_range(starting_date, ending_date, freq = 'MS')
#Create empty dataframe with date as index
df = pd.DataFrame(0, columns = ['City', 'Temperature', 'Actual_Lat', 'Actual_Lon', 'Nearest_Lat', 'Nearest_Lon', 'Country'], index = date_range)
#Give index name as 'Date'
df.index.names = ['Date']
print("Total records in Empty Dataframe before comparison = " , len(df))

#Get size of time variable for iteration during updating temperature
dt = np.arange(0, dataset.variables['time'].size)
print("Total records in time index before comparison = " , len(dt))

#Since number of dates is not equal in empty dataframe and netcdf file, we will extract the commomn date
#Extract time data from netcdf file
time_var = dataset.variables['time']
#Create new dataframe for time which will be used for conversion and comparison
time_df = pd.DataFrame(time_var[:], columns=['masked_date'])
#Convert masked date to 'cftime._cftime.DatetimeGregorian' in a new column
time_df['greg_date'] = nc.num2date(time_var[:], units)

#Function to convert cftime.DatetimeGregorian to pandas datetime
def convert_to_datetime(greg_date):
    return datetime(greg_date.year, greg_date.month, greg_date.day, greg_date.hour, greg_date.minute, greg_date.second)

#Convert 'cftime._cftime.DatetimeGregorian' date to pandas 'datetime'
#Apply the conversion function to the 'greg_date' column and add a new column 'converted_dates'
time_df['converted_dates'] = time_df['greg_date'].apply(convert_to_datetime)
#Change day from 15 to 1
time_df['converted_dates'] = time_df['converted_dates'].apply(lambda x: x.replace(day = 1))
#print(time_df.head())

#Check if dates present in NetCDF also exist in the empty dataFrame index i.e. get common dates
matching_dates = []
for date in time_df['converted_dates']:
    if date in df.index:
        matching_dates.append(date)
        
print("Total records in Matching list = " , len(matching_dates))

#Keep only matching dates (rows) in the empty dataframe
df = df[df.index.isin(matching_dates)]
print("Total records in Empty Dataframe after comparison = " , len(df))

#Initialize a dataframe. It will contain the final dataset
final_df = pd.DataFrame(columns = ['Date', 'City', 'Temperature', 'Actual_Lat', 'Actual_Lon', 'Nearest_Lat', 'Nearest_Lon', 'Country'])

#Extract Temperature data for each city and store it in 'final_df'
for cities in final_cities.city:
    
    #Get lat and lon for a specific city
    city_lat = final_cities.loc[final_cities['city'] == cities, 'lat'].iloc[0]
    city_lon = final_cities.loc[final_cities['city'] == cities, 'lng'].iloc[0]
    city_country = final_cities.loc[final_cities['city'] == cities, 'country'].iloc[0]

    #Calculate Squared distance between city location and location present in netcdf grid (i.e. choose the nearest point)
    sq_diff_lat = (lat - city_lat)**2
    sq_diff_lon = (lon - city_lon)**2

    #Get index of minimum value (i.e. smalleest squared distance)
    lat_index = sq_diff_lat.argmin()
    lon_index = sq_diff_lon.argmin()
    
    #Update temperature for specific city in empty dataframe
    for time_index in dt:
        df.iloc[time_index] = cities, temp[time_index, lat_index, lon_index], city_lat, city_lon, lat[lat_index], lon[lon_index], city_country
        
    #Create new dataframe to store result
    df1 = df.copy()
    #Convert index to column so that date can be a present as column
    df1.reset_index(inplace=True)
    #Concate all the data
    final_df = pd.concat([final_df, df1], axis=0, ignore_index=True)
    
print(final_df)

Total records in Empty Dataframe before comparison =  2681
Total records in time index before comparison =  1720
Total records in Matching list =  1720
Total records in Empty Dataframe after comparison =  1720
             Date            City  Temperature  Actual_Lat  Actual_Lon  \
0      1880-01-01  Belo Horizonte         0.00    -19.9150    -43.9150   
1      1880-02-01  Belo Horizonte         0.00    -19.9150    -43.9150   
2      1880-03-01  Belo Horizonte         0.00    -19.9150    -43.9150   
3      1880-04-01  Belo Horizonte         0.00    -19.9150    -43.9150   
4      1880-05-01  Belo Horizonte         0.00    -19.9150    -43.9150   
...           ...             ...          ...         ...         ...   
178875 2022-12-01          Kampot         0.80     10.6171    104.1833   
178876 2023-01-01          Kampot         0.65     10.6171    104.1833   
178877 2023-02-01          Kampot         1.20     10.6171    104.1833   
178878 2023-03-01          Kampot         0.71    